In [1]:
import os
import pandas as pd

import gtfs #homemade
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [2]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('2015-12-03','gtfs/')
# stops = gtfs.load_stops('2015-12-03','gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('2015-12-03','gtfs/')
print 'Finished loading GTFS data.'

Finished loading GTFS data.


In [3]:
trips.head()

,block_id,direction_id,route_id,service_id,shape_id,trip_headsign
trip_id,,,,,,
EN_U5-Weekday-008000_B12_1,NaN,1,B12,EN_U5-Weekday,B120102,LEFFRTS GDNS OCEAN AV
EN_U5-Weekday-011000_B12_1,NaN,0,B12,EN_U5-Weekday,B120101,EAST NY ALABAMA AV
EN_U5-Weekday-016000_B12_1,NaN,1,B12,EN_U5-Weekday,B120102,LEFFRTS GDNS OCEAN AV
EN_U5-Weekday-019000_B12_1,NaN,0,B12,EN_U5-Weekday,B120101,EAST NY ALABAMA AV
EN_U5-Weekday-024000_B12_1,NaN,1,B12,EN_U5-Weekday,B120102,LEFFRTS GDNS OCEAN AV


In [4]:
stop_times.head()

arrival_time departure_time  stop_sequence  \
trip_id                    stop_id                                              
EN_U5-Weekday-008000_B12_1 901471      01:20:00       01:20:00              1   
                           301386      01:21:28       01:21:28              2   
                           301387      01:22:00       01:22:00              3   
                           301388      01:22:55       01:22:55              4   
                           301389      01:23:13       01:23:13              5   

                                    pickup_type  drop_off_type  
trip_id                    stop_id                              
EN_U5-Weekday-008000_B12_1 901471             0              0  
                           301386             0              0  
                           301387             0              0  
                           301388             0              0  
                           301389             0              0

In [50]:
joined = stop_times.reset_index(level=1)[['stop_id','stop_sequence']].join(trips[['route_id','direction_id']])
joined.head()

,stop_id,stop_sequence,route_id,direction_id
trip_id,,,,
10004529-ECPD5-EC_S5-Weekday-20,150038,1,BX23,1
10004529-ECPD5-EC_S5-Weekday-20,101734,2,BX23,1
10004529-ECPD5-EC_S5-Weekday-20,101735,3,BX23,1
10004529-ECPD5-EC_S5-Weekday-20,103676,4,BX23,1
10004529-ECPD5-EC_S5-Weekday-20,101737,5,BX23,1


We don't need a row for every trip_id, so drop duplicates.  However there may be multiple combinations of stop_id and stop_sequence, so do not verify unique index yet.

In [48]:
unique_stops = joined.reset_index().drop_duplicates(subset=['route_id','direction_id','stop_id','stop_sequence']).drop('trip_id',axis=1)
unique_stops = unique_stops.set_index(['route_id','stop_id'],verify_integrity=False)

Show which route_id + stop_id indices have multiple rows of stop_sequence or direction_id

In [49]:
dupe_inds = unique_stops.index[unique_stops.index.duplicated()]
unique_stops.loc[dupe_inds]

stop_sequence  direction_id
route_id stop_id                             
BX23     150139              24             1
         150139               1             0
         150139               2             1
         150038               1             1
         150038              23             0
Q50      101734              27             0
         101734               1             1
BXM9     103561               1             1
         103561              19             0
BXM7     103674               1             1
         103674              14             0
BXM6     103294              11             0
         103294               1             1
BXM4     150151               1             1
         150151              30             0
Q37      551203              15             0
         551203              16             0
         551203               2             0
         551205              16             0
         551205              17             0
         551205               3             0
         551207              17             0
         551207              18             0
         551207               4             0
         551211              18             0
         551211              19             0
         551211               5             0
         551213              19             0
         551213              20             0
         551213               6             0
...                         ...           ...
S44      200858               5             0
         200858              52             0
         200858              48             0
         200040               6             0
         200040              53             0
         200040              49             0
         200041               7             0
         200041              54             0
         200041              50             0
         200042               8             0
         200042              55             0
         200042              51             0
         200043               9             0
         200043              56             0
         200043              52             0
         200044              10             0
         200044              57             0
         200044              53             0
         200045              11             0
         200045              58             0
         200045              54             0
         200046              12             0
         200046              59             0
         200046              55             0
         200047              13             0
         200047              60             0
         200047              56             0
         805016              14             0
         805016              61             0
         805016              57             0

[63226 rows x 2 columns]